In [7]:
"""
使用的是最开始的那个简单的MNIST版本神经网络
"""

import tensorflow as tf
from tensorflow.examples.tutorials import input_data
import numpy as np

mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

batch_size = 100

n_batch = mnist.train.num_examples // batch_size


def variable_summaries(var):
    """
    传入一个参数var,这里计算这个参数的平均值，标准差，最大值，最小值，直方图
    当需要分析哪一个参数的时候，就在哪里调用这个函数
    """
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        # 平均值
        """ tf.summary.scalar 函数的第一个参数是给最后求的的起名字 """
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        # 标准差
        tf.summary.scalar('stddev', stddev)
        # 最大值
        tf.summary.scalar('max', tf.reduce_max(var))
        # 最小值
        tf.summary.scalar('min', tf.reduce_max(var))
        # 直方图
        tf.summary.scalar('histogram', var)


"""
要可视化神经网络的结构首先要定义一个命名空间，传入的参数是这个命名空间的名字，可以自己修改
"""
with tf.name_scope('input'):
    """ 注意，这里的x和y,都要放在这个命名空间的下面, 分别给x,y取名字，为了后来画图方便"""
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y = tf.placeholder(tf.float32, [None, 10], name='y-input')

# 创建一个简单的神经网络，只有输入层和有输出层， 没有隐藏层；
# 再定义一个命名空间，将参数W和b都放在这个命名空间下面
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784, 10]), name='w')
        """ 比如当我们需要分析在训练过程中权值W的变化时，我们就可以调用 variable_summaries(var) 了"""
        variable_summaries(W)
        
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]), name='b')
        variable_summaries(b)
        
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x, W) + b
        
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)
        
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = tf.matmul(x, W) + b))
    """ 这里在考虑损失函数的时候，因为这里loss只有1个值，所以计算loss的平均值，最大，最小都没有意义，所以这里直接就  """
#     variable_summaries(loss)
    tf.summary.scalar('loss', loss)
    
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

# 初始化就不用起名字了，初始化默认就有名字叫做init
init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope('accu'):
        # 求准确率, tf.cast 就是类型转换
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

        
# 将前面检测的所有指标都合并起来，合并所有的Summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    """ 第一个参数是保存文件的路径，第二个参数表示存放的是graph,也就是图的结构 """
    writer = tf.summary.FileWriter('logs/', sess.graph)
    for epoch in range(51):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
            # 获取summary
            # 横线_作做变量名。run里面fetch两步，对应两步的结果拿两个变量来存储
            # 因为sess.run  运行的是merger 和train_step ,返回的是一个列表，包含两个值
            summary, _ = sess.run([merged,train_step], feed_dict={x:batch_xs, y:batch_ys})
        
        # 将得到的summary 写入到生成的文件中,每执行一个周期，都写入这个周期的summary
        writer.add_summary(summary, epoch)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("Iter", str(epoch), "Testing Accuracy = ", str(acc))
        writer.close()
        writer.close()


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


InvalidArgumentError: You must feed a value for placeholder tensor 'input/x-input' with dtype float and shape [?,784]
	 [[Node: input/x-input = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'input/x-input', defined at:
  File "/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/anaconda2/lib/python2.7/site-packages/tornado/ioloop.py", line 1073, in start
    handler_func(fd_obj, events)
  File "/anaconda2/